# Tips Prediction #

In this library, we will explore the data which we retrieved from *LifeProTips*, *ShittyLifeProTips*, *UnethicalLifeProTips* and *IllegalLifeProTips*.
Afterwards, we will make some predictions to determine which subreddit a certain post belongs to.

In [ ]:
# Import everything in case we just want to run particular cells
import json
import math
import pandas as pd
import re
from string import punctuation
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Load and split ##

Let's start by loading the posts into memory

In [ ]:
import json

posts = {}

# Import LifeProTips
with open('tips/lifeprotips_dump_1.json', 'r') as lptf:
    posts['lpt'] = json.load(lptf)

# Import ShittyLifeProTips
with open('tips/shittylifeprotips_dump_1.json', 'r') as slptf:
    posts['slpt'] = json.load(slptf)
    
# Import UnethicalLifeProTips
with open('tips/unethicallifeprotips_dump_1.json', 'r') as ulptf:
    posts['ulpt'] = json.load(ulptf)

# Import IllegalLifeProTips
with open('tips/illegallifeprotips_dump_1.json', 'r') as ilptf:
    posts['ilpt'] = json.load(ilptf)

Let's remove any duplicate from the posts

In [ ]:
for subreddit in posts:
    obs_ids = [] # observed ids
    unique_posts = [] # unique posts
    for post in posts[subreddit]: # every post is a dictionary
        if post['id'] not in obs_ids:
            obs_ids.append(post['id'])
            unique_posts.append(post)
    posts[subreddit] = unique_posts

Check the amount of unique posts we retrieved per subreddit $\rightarrow$ No duplicates!

In [ ]:
for (s, a) in posts.items():
    print('{} posts retrieved from {}'.format(len(a), s))

The reason why there are only 10k posts from *IllegalLifeProTips* is because the community is relatively young compared to the rest of subreddits. It was founded a year ago while the other three appeared 7 years ago.

---

Now, let's shuffle the posts for each subreddit to perform an arbitrary and even train/validation/test division

In [ ]:
import random

for (s, p) in posts.items():
    random.shuffle(p)

We will perfom a train/validation/test split as follows:
* $4/9$ for training ($2/3$ of $2/3$)
* $2/9$ for validation ($1/3$ of $2/3$)
* $1/3$ for testing

In [ ]:
import math

# Create and empty dictionary to store train, validation and test posts
posts_struct = {'train':[], 'validation':[], 'test':[]}

# For each subreddit, split
for p in posts.values():
    # p is the list of posts (each post is a dictionary)
    # train + validation is 2/3; test is 1/3
    split_index = int(math.ceil((2/3)*len(p)))
    p_trainval = p[:split_index]
    p_test = p[split_index:]
    # Now, out of train+validation, 2/3 go to train and 1/3 to validation
    split_index = int(math.ceil((2/3)*len(p_trainval)))
    p_train = p_trainval[:split_index]
    p_validation = p_trainval[split_index:]
    
    # Finally, add the posts to their respective parts in the posts_struct dictionary
    posts_struct['train'].extend(p_train)
    posts_struct['validation'].extend(p_validation)
    posts_struct['test'].extend(p_test)

# Shuffle the lists
for (sep, p) in posts_struct.items():
    random.shuffle(p)

Check that we performed a correct split

In [ ]:
ins_tot = len(posts_struct['train']) + len(posts_struct['validation']) + len(posts_struct['test'])
print('Total number of instances: {}'.format(ins_tot))
print('Percentage of train instances: {}'.format((len(posts_struct['train'])/ins_tot)*100))
print('Percentage of validation instances: {}'.format((len(posts_struct['validation'])/ins_tot)*100))
print('Percentage of test instances: {}'.format((len(posts_struct['test'])/ins_tot)*100))

We just need one more step to perform. Creating `pandas` DataFrames for training, validation and testing will make data handling easier

In [ ]:
import pandas as pd

# DataFrame for training
df_train = pd.DataFrame({'instance':[s['title'] for s in posts_struct['train']],
                         'label':[s['label'] for s in posts_struct['train']]})

# DataFrame for validation
df_validation = pd.DataFrame({'instance':[s['title'] for s in posts_struct['validation']],
                              'label':[s['label'] for s in posts_struct['validation']]})

# DataFrame for testing
df_test = pd.DataFrame({'instance':[s['title'] for s in posts_struct['test']],
                        'label':[s['label'] for s in posts_struct['test']]})

## 2. Instance pre-processing ##

Define a method to clean the instances<br>
**Disclaimer:** we are **not** copying this from Assignment 2

In [ ]:
def clean(text, stem_words=True):
    
    # Empty question
    if type(text) != str or text=='':
        return ''
    
    # Make text lowercase
    text = text.lower()

    # Clean the text
    text = re.sub("\'s", " ", text) # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub("\'t'", " not ", text)
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub(" thats ", " that is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("-", " ", text)
    #you might need more
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    
    # remove punctuation
    for c in punctuation:
        text = re.sub("\\" + c, "", text)
        
    # split text
    n_text = text.split();
    
    # remove stopwords and subreddit keywords
    stops = set(['the', 'to', 'now', 'should', 'just', 'very', 'too', 'again', 'in', 'out', 'on',
                 'over', 'at', 'of', 'about'])
    #stops = set(stopwords.words('english'))
    sub_kwords = set(['lpt', 'ulpt', 'ilpt', 'slpt'])
    numbers = set(['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve'])
    n_text = [word for word in n_text if word not in stops|sub_kwords|numbers if word.isalpha()]
    
    # Change numbers and other similar words
    #subs = [('one', '1'), ('ii', '2'), ('two', '2'), ('iii', '3'), ('three', '3'), ('iv', '4'), ('four', '4'),
    #        ('v', '5'), ('five', '5'), ('vi', '6'), ('six', '6'), ('vii', '7'), ('seven', '7'), ('viii', '8'),
    #        ('eight', '8'), ('ix', '9'), ('nine', '9'), ('ten', '10'), ('eleven', '11'), ('twelve', '12')]
    #for i, word in enumerate(n_text):
    #    for (bad, good) in subs:
    #        if word == bad:
    #            n_text[i] = good
    #            break
    
    # Return a list of words
    return ' '.join(n_text)

Clean the DataFrames

In [ ]:
df_train['instance'] = df_train['instance'].apply(clean)
df_validation['instance'] = df_validation['instance'].apply(clean)
df_test['instance'] = df_test['instance'].apply(clean)

# Drop empty posts
df_train = df_train[df_train['instance'] != ""]
df_train = df_train[df_train['instance'] != " "]
df_validation = df_validation[df_validation['instance'] != ""]
df_validation = df_validation[df_validation['instance'] != " "]
df_test = df_test[df_test['instance'] != ""]
df_test = df_test[df_test['instance'] != " "]

## 3. Train the ML Models ##

### BOW + Naive Bayes ###

First, build the BOW representation for training, validation and testing

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# Generate the vocabulary using all instances
vectorizer.fit(np.concatenate((df_train['instance'].values, df_validation['instance'].values,
                               df_test['instance'].values), axis=0))

# Transform each batch separately and add labels
# Train
x_train = vectorizer.transform(df_train['instance'].values).toarray()
y_train = df_train['label'].values
# Validation
x_validation = vectorizer.transform(df_validation['instance'].values).toarray()
y_validation = df_validation['label'].values
# Train+Validation
x_tv = np.concatenate((x_train, x_validation), axis=0)
y_tv = np.concatenate((y_train, y_validation), axis=0)
# Test
x_test = vectorizer.transform(df_test['instance'].values).toarray()
y_test = df_test['label'].values

Perform validation using `MultinomialNB` from `sklearn`

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import MultinomialNB

alphas = np.linspace(0.3,0.8,10)
accs = []
precs = []
recs = []
fscores = []

top_acc = 0
top_alpha = 0
for alpha in alphas:
    model = MultinomialNB(alpha)
    model.fit(x_train, y_train)
    acc = model.score(x_validation, y_validation)
    accs.append(acc)
    measures = precision_recall_fscore_support(y_validation, model.predict(x_validation), average='macro')
    precs.append(measures[0])
    recs.append(measures[1])
    fscores.append(measures[2])
    if acc > top_acc:
        top_acc = acc
        top_alpha = alpha
print("Top accuracy:", top_acc)
print("Alpha:", top_alpha)

# Retrain with whole training set and optimal alpha
top_model = MultinomialNB(top_alpha)
top_model.fit(x_tv, y_tv);

Plot measures vs alpha

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

# Accuracy vs alpha
axes[0][0].plot(alphas, accs, lw=1.5)
axes[0][0].set_xlabel(r'$\alpha$')
axes[0][0].set_ylabel('Accuracy')
axes[0][0].set_title('Accuracy vs. Alpha for MultinomialNB')

# Precision vs alpha
axes[0][1].plot(alphas, precs, lw=1.5)
axes[0][1].set_xlabel(r'$\alpha$')
axes[0][1].set_ylabel('Precision')
axes[0][1].set_title('Precision vs. Alpha for MultinomialNB')

# Recall vs alpha
axes[1][0].plot(alphas, recs, lw=1.5)
axes[1][0].set_xlabel(r'$\alpha$')
axes[1][0].set_ylabel('Recall')
axes[1][0].set_title('Recall vs. Alpha for MultinomialNB')

# FScore vs alpha
axes[1][1].plot(alphas, fscores, lw=1.5)
axes[1][1].set_xlabel(r'$\alpha$')
axes[1][1].set_ylabel('FScore')
axes[1][1].set_title('FScore vs. Alpha for MultinomialNB')

# Adjust layout
fig.tight_layout()

Check for accuracy on test set

In [ ]:
print("MN Accuracy: %0.2f%%" % (100 * top_model.score(x_test, y_test)))

Compute confusion matrix, precision, recall and F-score with micro and macro averaging

In [ ]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(y_test, top_model.predict(x_test))

measures_macro = list(precision_recall_fscore_support(y_test, top_model.predict(x_test), average='macro'))[:-1]
measures_micro = list(precision_recall_fscore_support(y_test, top_model.predict(x_test), average='micro'))[:-1]

Show confusion matrix

In [ ]:
cf_df = pd.DataFrame(cf, columns=['Predicted ilpt', 'Predicted lpt', 'Predicted slpt', 'Predicted ulpt'])
cf_df.rename(index={0:'ilpt',1:'lpt',2:'slpt',3:'ulpt'}, inplace=True)
cf_df

Show precision, recall and F-score with averaging

In [ ]:
measures_macro = ['macro'] + measures_macro
measures_micro = ['micro'] + measures_micro
measures_df = pd.DataFrame([measures_macro, measures_micro], columns=['Averaging', 'Precision', 'Recall', 'FScore'])
measures_df

### Word2Vec ###

We will train a `Word2Vec` model from `gensim` using our entire corpus.

First, we define a function to tokenize a list of strings

In [ ]:
def tokenize_arr(sentences):
    """Given a list of sentences, return the list with the tokenized strings (a list itself)."""
    new_sentences = sentences.copy()
    for i, sentence in enumerate(new_sentences):
        new_sentences[i] = sentence.split()
    return new_sentences

Generate the `sentences` for `Word2Vec` using all corpus

In [ ]:
sentences = tokenize_arr(np.concatenate((df_train['instance'].values,
                                         df_validation['instance'].values,
                                         df_test['instance'].values), axis=0))

Train `Word2Vec` using `sentences`.<br>
We must look for good parameter values regarding `min_count`, `size` and `window`.

In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences, min_count=1, size=10, window=1)

Embeddings must be generated for the instances in `df_train`, `df_validation` and `df_test`

In [ ]:
# Tokenize the submissions in train, validation and test
sub_train = tokenize_arr(df_train['instance'].values)
sub_validation = tokenize_arr(df_validation['instance'].values)
sub_test = tokenize_arr(df_test['instance'].values)

Let's define a method to transform a list of tokens into an embedding

In [ ]:
def sum_lists(lol):
    """Given a list of lists of equal size, sum its elements (like vector sum) and take the average."""
    result = []
    for element in lol[0]: # Create the size of sum array
        result.append(0)
    for l in lol: # For every sublist
        for i, c in enumerate(l): # For every element
            result[i] += c
    # Average the sum
    for i, v in enumerate(result):
        result[i] /= len(lol);
    return result

In [ ]:
def sen2emb(sentence, word_dict):
    """Transform a tokenized sentence into an embedding by adding the embedding word vectors."""
    embeddings = []
    for token in sentence:
        if token in word_dict.vocab:
            embeddings.append(word_dict[token].tolist())
    embedding = sum_lists(embeddings)
    return embedding

Generate $x$ for training, validation and testing

In [ ]:
x_train = [sen2emb(sub, w2v_model.wv) for sub in sub_train]
x_validation = [sen2emb(sub, w2v_model.wv) for sub in sub_validation]
x_tv = np.concatenate((x_train, x_validation), axis=0)
x_test = [sen2emb(sub, w2v_model.wv) for sub in sub_test]

Generate the labels for training, validation and testing

In [ ]:
y_train = df_train['label'].values
y_validation = df_validation['label'].values
y_tv = np.concatenate((y_train, y_validation), axis=0)
y_test = df_test['label'].values

Perform **feature standarization** so that the values of each feature in the data have zero-mean and unit-variance.

$$x'=\frac{x-\overline{x}}{\sigma}$$

Where $x$ is the original feature vector, $\overline{x} = average(x)$ is the mean of that feature vector, $\sigma$ is its standard deviation.

In [ ]:
def standarize(lol):
    """Given a list of list, standarize the sublists."""
    lol_np = np.array(lol)
    avg_v = np.mean(lol_np, axis=0)
    std_v = np.std(lol_np, axis=0)
    return (lol_np - avg_v) / std_v

Standarize $x$ for training, validation and testing

In [ ]:
x_train = standarize(x_train)
x_validation = standarize(x_validation)
x_tv = standarize(x_tv)
x_test = standarize(x_test)

#### Support Vector Machine ####

Tune hyperparameters and retrieve measures

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support

top_accuracy = 0;
top_reg = 0;

regs = np.linspace(1e-10, 1, 10)
accs = []
precs = []
recs = []
fscores = []

for reg in regs:
    svm_model = SVC(gamma='scale', C=reg)
    svm_model.fit(x_train, y_train)
    acc = svm_model.score(x_validation, y_validation)
    accs.append(acc)
    measures = precision_recall_fscore_support(y_validation, svm_model.predict(x_validation), average='macro')
    precs.append(measures[0])
    recs.append(measures[1])
    fscores.append(measures[2])
    if acc > top_accuracy:
        top_accuracy = acc
        top_reg = reg

print("Top regularization:", top_reg)
print("Top accuracy:", top_accuracy)

# Retrain with whole training set and optimal penalty
top_model = SVC(gamma='scale', C=top_reg)
top_model.fit(x_tv, y_tv);

Plot measures vs. penalty

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

# Accuracy vs regularization
axes[0][0].plot(regs, accs, lw=1.5)
axes[0][0].set_xlabel('Regularization')
axes[0][0].set_ylabel('Accuracy')
axes[0][0].set_title('Accuracy vs. Regularization for SVM')

# Precision vs regularization
axes[0][1].plot(regs, precs, lw=1.5)
axes[0][1].set_xlabel('Regularization')
axes[0][1].set_ylabel('Precision')
axes[0][1].set_title('Precision vs. Regularization for SVM')

# Recall vs regularization
axes[1][0].plot(regs, recs, lw=1.5)
axes[1][0].set_xlabel('Regularization')
axes[1][0].set_ylabel('Recall')
axes[1][0].set_title('Recall vs. Regularization for SVM')

# FScore vs regularization
axes[1][1].plot(regs, fscores, lw=1.5)
axes[1][1].set_xlabel('Regularization')
axes[1][1].set_ylabel('FScore')
axes[1][1].set_title('FScore vs. Regularization for SVM')

# Adjust layout
fig.tight_layout()

Perform test on test set

In [ ]:
print("SVM Accuracy: %0.2f%%" % (100 * top_model.score(x_test, y_test)))

Compute confusion matrix, precision, recall and F-score with micro and macro averaging

In [ ]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(y_test, top_model.predict(x_test))

measures_macro = list(precision_recall_fscore_support(y_test, top_model.predict(x_test), average='macro'))[:-1]
measures_micro = list(precision_recall_fscore_support(y_test, top_model.predict(x_test), average='micro'))[:-1]

Show the confusion matrix

In [ ]:
cf_df = pd.DataFrame(cf, columns=['Predicted ilpt', 'Predicted lpt', 'Predicted slpt', 'Predicted ulpt'])
cf_df.rename(index={0:'ilpt',1:'lpt',2:'slpt',3:'ulpt'}, inplace=True)
cf_df

Show precision, recall and F-score with averaging

In [ ]:
measures_macro = ['macro'] + measures_macro
measures_micro = ['micro'] + measures_micro
measures_df = pd.DataFrame([measures_macro, measures_micro], columns=['Averaging', 'Precision', 'Recall', 'FScore'])
measures_df

#### Logistic Regression ####

Tune hyperparameters and retrieve measures

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

top_accuracy = 0;
top_reg = 0;

regs = np.linspace(1e-10, 0.05, 200)
accs = []
precs = []
recs = []
fscores = []

for reg in regs:
    lr_model = LogisticRegression(solver='lbfgs', multi_class='multinomial', C=reg, max_iter=5000)
    lr_model.fit(x_train, y_train)
    acc = lr_model.score(x_validation, y_validation)
    accs.append(acc)
    measures = precision_recall_fscore_support(y_validation, lr_model.predict(x_validation), average='macro')
    precs.append(measures[0])
    recs.append(measures[1])
    fscores.append(measures[2])
    if acc > top_accuracy:
        top_accuracy = acc
        top_reg = reg

print("Top regularization:", top_reg)
print("Top accuracy:", top_accuracy)

# Retrain with whole training set and optimal penalty
top_model = LogisticRegression(solver='lbfgs', multi_class='multinomial', C=top_reg, max_iter=10000)
top_model.fit(x_tv, y_tv);

Plot measures vs. penalty

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

# Accuracy vs regularization
axes[0][0].plot(regs, accs, lw=1.5)
axes[0][0].set_xlabel('Regularization')
axes[0][0].set_ylabel('Accuracy')
axes[0][0].set_title('Accuracy vs. Regularization for LR')

# Precision vs regularization
axes[0][1].plot(regs, precs, lw=1.5)
axes[0][1].set_xlabel('Regularization')
axes[0][1].set_ylabel('Precision')
axes[0][1].set_title('Precision vs. Regularization for LR')

# Recall vs penalty
axes[1][0].plot(regs, recs, lw=1.5)
axes[1][0].set_xlabel('Regularization')
axes[1][0].set_ylabel('Recall')
axes[1][0].set_title('Recall vs. Regularization for LR')

# FScore vs regularization
axes[1][1].plot(regs, fscores, lw=1.5)
axes[1][1].set_xlabel('Regularization')
axes[1][1].set_ylabel('FScore')
axes[1][1].set_title('FScore vs. Regularization for LR')

# Adjust layout
fig.tight_layout()

Perform test on test set

In [ ]:
print("LR Accuracy: %0.2f%%" % (100 * top_model.score(x_test, y_test)))

Compute confusion matrix, precision, recall and F-score with micro and macro averaging

In [ ]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(y_test, top_model.predict(x_test))

measures_macro = list(precision_recall_fscore_support(y_test, top_model.predict(x_test), average='macro'))[:-1]
measures_micro = list(precision_recall_fscore_support(y_test, top_model.predict(x_test), average='micro'))[:-1]

Show the confusion matrix

In [ ]:
cf_df = pd.DataFrame(cf, columns=['Predicted ilpt', 'Predicted lpt', 'Predicted slpt', 'Predicted ulpt'])
cf_df.rename(index={0:'ilpt',1:'lpt',2:'slpt',3:'ulpt'}, inplace=True)
cf_df

Show precision, recall and F-score with averaging

In [ ]:
measures_macro = ['macro'] + measures_macro
measures_micro = ['micro'] + measures_micro
measures_df = pd.DataFrame([measures_macro, measures_micro], columns=['Averaging', 'Precision', 'Recall', 'FScore'])
measures_df

### Pretrained Word2Vec ###